Check that colab is using a GPU then install dependencies

In [ ]:
import torch
import os

print(f"✓ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✓ GPU: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ No GPU found - using CPU (will be slow)")

# Install required packages
print("\nInstalling packages...")
!pip install -q nnunetv2 SimpleITK nibabel numpy-stl scikit-image

# Create input directory
os.makedirs('/content/input', exist_ok=True)
os.makedirs('/content/output', exist_ok=True)

print("✓ Setup complete!")

Install models from hugging face and the nnu-net inference models

In [ ]:
##Huggingface model download
!pip install -q huggingface_hub
from huggingface_hub import snapshot_download
snapshot_download(repo_id="aagatti/nnunet_knee", local_dir="./nnunet_knee_models")

# Clone inference repository
!git clone https://github.com/gattia/nnunet_knee_inference.git /content/nnunet_knee_inference


Upload MRI files from computer (.nii format)

In [ ]:
from google.colab import files
import shutil

print("📤 Please select your knee_001.nii file to upload:")
uploaded = files.upload()

# Move uploaded file to input directory
for filename in uploaded.keys():
    shutil.move(filename, f'/content/input/{filename}')
    print(f"✓ Uploaded: {filename}")


In [ ]:
import sys
sys.path.insert(0, '/content/nnunet_knee_inference')

from scripts.inference import KneeSegmentationInference

# Initialize the inference
print("Loading model...")
inference = KneeSegmentationInference(
    model_dir="/content/nnunet_knee_model"
)

# Get the input file name
input_files = os.listdir('/content/input')
if len(input_files) == 0:
    print("❌ No input files found!")
else:
    input_file = input_files[0]
    input_path = f'/content/input/{input_file}'
    output_path = f'/content/output/{input_file.replace(".nii", "_seg.nii.gz")}'
    
    print(f"Processing: {input_file}")
    print("⏳ This will take 5-10 minutes with GPU, 20-30 minutes with CPU...")
    
    # Run prediction
    result = inference.predict(input_path, output_path)
    
    print(f"✓ Segmentation complete!")
    print(f"Output saved to: {output_path}")

In [ ]:
from google.colab import files

# List output files
print("Available output files:")
output_files = os.listdir('/content/output')
for f in output_files:
    print(f"  - {f}")

# Download the segmentation
if len(output_files) > 0:
    output_file = output_files[0]
    output_path = f'/content/output/{output_file}'
    
    print(f"\n📥 Downloading: {output_file}")
    files.download(output_path)
    print("✓ Download complete!")
else:
    print("❌ No output files found")